In [1]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai

categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

In [4]:
print(sports_dataset['data'][0])
#print(sports_dataset)

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [10]:
sports_dataset.target_names[sports_dataset['target'][0]]
#sports_dataset.target_names[0]

'rec.sport.baseball'

In [11]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")
print(len(sports_dataset.target))

Total examples: 1197, Baseball examples: 597, Hockey examples: 600
1197


In [10]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df = df.head(300)
print(df)


                                                prompt completion
0    From: dougb@comm.mot.com (Doug Bank)\nSubject:...   baseball
1    From: gld@cunixb.cc.columbia.edu (Gary L Dare)...     hockey
2    From: rudy@netcom.com (Rudy Wade)\nSubject: Re...   baseball
3    From: monack@helium.gas.uug.arizona.edu (david...     hockey
4    Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....   baseball
..                                                 ...        ...
295  From: poutsmaj@mace.cc.purdue.edu (unknown)\nS...   baseball
296  From: kkeller@mail.sas.upenn.edu (Keith Keller...     hockey
297  From: mtt@kepler.unh.edu (Matthew T Thompson)\...   baseball
298  From:  (Austin Jacobs)\nSubject: BOB KNEPPER W...   baseball
299  From: umturne4@ccu.umanitoba.ca (Daryl Turner)...     hockey

[300 rows x 2 columns]


In [11]:
df.to_json("sport2.jsonl", orient='records', lines=True)

In [12]:
!pip install --upgrade openai

In [13]:
!openai tools fine_tunes.prepare_data -f sport2.jsonl -q

Analyzing...

- Your file contains 300 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 3 examples that are very long. These are rows: [134, 200, 281]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whi

In [ ]:
!openai api fine_tunes.list

{
  "data": [
    {
      "created_at": 1678547617,
      "fine_tuned_model": "ada:ft-personal-2023-03-11-15-30-13",
      "hyperparams": {
        "batch_size": 1,
        "classification_positive_class": " baseball",
        "compute_classification_metrics": true,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-oKMnEhQkmQ3WY3HftJibZzxO",
      "model": "ada",
      "object": "fine-tune",
      "organization_id": "org-94IiGgQZbJYNSQZpYNoq7mL1",
      "result_files": [
        {
          "bytes": 214954,
          "created_at": 1678548615,
          "filename": "compiled_results.csv",
          "id": "file-2zUGVEepuk536mYVjcqL6wdk",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 1519036,
          "created_at": 1678547615

In [ ]:
!openai api fine_tunes.create -t "sport2_prepared_train.jsonl" -v "sport2_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " baseball" -m ada

In [1]:
!openai api fine_tunes.list

{
  "data": [
    {
      "created_at": 1678547617,
      "fine_tuned_model": "ada:ft-personal-2023-03-11-15-30-13",
      "hyperparams": {
        "batch_size": 1,
        "classification_positive_class": " baseball",
        "compute_classification_metrics": true,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-oKMnEhQkmQ3WY3HftJibZzxO",
      "model": "ada",
      "object": "fine-tune",
      "organization_id": "org-94IiGgQZbJYNSQZpYNoq7mL1",
      "result_files": [
        {
          "bytes": 214954,
          "created_at": 1678548615,
          "filename": "compiled_results.csv",
          "id": "file-2zUGVEepuk536mYVjcqL6wdk",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 1519036,
          "created_at": 1678547615

In [2]:
!openai api fine_tunes.results -i ft-oKMnEhQkmQ3WY3HftJibZzxO > result_20230316.csv

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2800.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2800.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\dougg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\Scripts\openai.exe\__main__.py", line 7, in <module>
  File "C:\Users\dougg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openai\_openai_scripts.py", line 63, in main
    args.func(args)
  File "C:\Users\dougg\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openai\cli.py", line 497, in results
    print(res